<center><img src="media/imagenes/Banner.png" height = 100 width = 1000></center>

Este notebook pretende enseñar el resultadod de todas las implementaciones sin meterse mucho en los tedalles de la implementación, pretende ser una guia visual e interactiva, para endender todas las decisiones de diseño que hemos tomado en el proyecto. Esta pensado para ser ejecutado celda a celda salvo que se diga lo contrario.La idea

# La idea

In [ ]:
import dependencias
from pprint import pprint

from dependencias.Agentes import Cycle_detector

## Agentes

<img src="media/imagenes/euristicos.png" style="float: right;">

### Agentes Basados en Heuristicas.

Los agentes que vamos a enseñar en este apartado son agentes que se basan en una politica de recompensa para decidir que movimiento realizar en cada turno, estan implementados mediante una clase abstracta que tiene los siguientes metodos:

- get_reward: Calcula la recompensa asociada a cada movimiento en funcion de una politica de recompensa.
- get_action: Calcula la mejor acción a realizar en función de la recompensa obtenida.

de este modo podemos empezar a jugar con el agente y ver como se comporta.



A continuacion vamos a enseñar los agentes que hemos implementado y como se comportan en el juego.

## Agentes Chaser

La politica de recompensa de este agente se basa en la distancia entre la cabeza de la serpiente y la comida mas cercana, de este modo el agente siempre intentará acercarse a la comida, aunque esto implique que la serpiente se muera.

<center><img src="media/imagenes/chasser.png" style="float: center;"></center>

In [64]:
agente_chaser = dependencias.Agentes.ChaserAgent()

In [65]:
# Ejecuta esta celda para ver como se comporta el agente puedes modificar los parametros del juego para ver como se comporta en distintos escenarios

SIZE = (15, 15)
N_FOODS = 5

game = dependencias.Snake_game(SIZE, N_FOODS, agente_chaser)
game.play_with_pygame()

In [66]:
estadisticas = game.evaluar()
pprint(estadisticas)

100%|██████████| 100/100 [00:00<00:00, 1522.94it/s]

{'movimientos por puntuacion': 4,
 'movimientos_maximos': np.int64(115),
 'movimientos_medios': np.float64(29.64),
 'movimientos_minimos': np.int64(6),
 'proporcion_del_tablero_ocupada': 0.01,
 'puntuacion_maxima': np.int64(23),
 'puntuacion_media': np.float64(6.81),
 'puntuacion_minima': np.int64(3)}


### Conclusiones

Este agente logra su objetivo de llegar muy rapido a por la comida (movimientos por puntuacion), pero no consigue sobrevivir mucho tiempo.

### Mejoras

Este agente podría ser mejorado si se le añade una penalización por muerte inmediata.

## Agentes Avoider

La politica de recompensa de este agente se basa en evitar las muertes inediatas, ya sea contra la serpriente o contra las paredes, asociando una penalización a estos movimientos.

<center><img src="media/imagenes/inminent.png" style="float: center;"></center>

In [67]:
agente_avoider = dependencias.Agentes.Avoid_inmediate_death()

In [ ]:
# Ejecuta esta celda para ver como se comporta el agente
SIZE = (15, 15)
N_FOODS = 5

game = dependencias.Snake_game(SIZE, N_FOODS, agente_avoider)
game.play_with_pygame()

In [9]:
estadisticas = game.evaluar()
pprint(estadisticas)

100%|██████████| 100/100 [00:00<00:00, 367.70it/s]

{'movimientos por puntuacion': 63,
 'movimientos_maximos': np.int64(1379),
 'movimientos_medios': np.float64(634.02),
 'movimientos_minimos': np.int64(175),
 'proporcion_del_tablero_ocupada': 0.04,
 'puntuacion_maxima': np.int64(17),
 'puntuacion_media': np.float64(10.13),
 'puntuacion_minima': np.int64(5)}


### Conclusiones

Este agente majora mucho su tiempo de vida, pero no consigue alcanzar la comida de manera eficiente.

### Mejoras

Ademas vemos que con evitar muertes inmediatas no es suficiente, puesto que hay escenarios en los que te puedes quedar encerrado.



## Combinacion de agentes

Esta implementation de los agentes tan particular, nos permite combinar las recompensas que le dan a cada movimiento distintos agentes para conseguir un agente que se comporte de una manera más eficiente.

<img src="media/imagenes/combinado.png" style="display: block; margin: auto;" width="400">



In [10]:
chouder = dependencias.Agentes.Combined_agent((agente_avoider, agente_chaser), (1, 0.5))

In [11]:
# Ejecuta esta celda para ver como se comporta el agente
game = dependencias.Snake_game((15, 15), 5, chouder)
game.play_with_pygame()

In [12]:
estadisticas = game.evaluar()
pprint(estadisticas)

100%|██████████| 100/100 [00:00<00:00, 207.17it/s]

{'movimientos por puntuacion': 5,
 'movimientos_maximos': np.int64(401),
 'movimientos_medios': np.float64(179.84),
 'movimientos_minimos': np.int64(34),
 'proporcion_del_tablero_ocupada': 0.14,
 'puntuacion_maxima': np.int64(73),
 'puntuacion_media': np.float64(33.59),
 'puntuacion_minima': np.int64(10)}


### Conclusiones

El comportaiento de este agente es mucho mejor que el de los agentes individuales, y parece una buena estrategia para el inicio del juego.

### Mejoras

El agente sigue sin ser capaz de evitar quedarse encerrado, hay que añadir una heuristica mas a futuro para evitar este tipo de situaciones.

<img src="media/imagenes/encerrado.png" style="display: block; margin: auto;" width="400">

*en el ejemplo moverse a la derecha provoca una muerte inevitable pero el agente no lo considera una mala jugada*

## Politicas basadas en busqueda en anchura (Estrategias a largo plazo)

para hacer decisiones a largo plazo se nos plantean dos opciones.

- Fuerza bruta (Exploracion en estados): podemos ver a donde nos lleva una combinacion de movimientos y ver cual es mejor.

problema: el espacio de estados es muy grande y no podemos explorar todas las posibilidades, tendriamos que seleccionar una profundidad maxima y explorar hasta ahi. Además relentizaria cada simulacion.

- Busqueda en anchura: Lanzando busqueda en anchura desde los movimientos posibles en cada turno podemos ver cual es el mejor movimiento a largo plazo, evaluando algunas caracteristicas del tablero.

A continuacion desarrollamos agentes basados en busqueda en anchura, para solucionar probleas de muerte a largo plazo.

## Agentes Searcher

Este agente lanza 3 busquedas en anchura, empezando en cada una las casillas accedibles en el siguiente movimiento, aprovecha para contar el numer de casillas accesibles en cada caso y asigna una puntuacion a cada movimiento en funcion de este valor.

<div style="text-align: center;">
$$
W = \frac{\text{casillas accesibles en el movimiento } m}{\text{total casillas accesibles}}
$$
</div>

*De este modo premiamos los movimientos que te llevan a los espacios abiertos*


<img src="<img src="media/imagenes/searcher.png" style="display: block; margin: auto;" width="400">

In [26]:
agente_searcher = dependencias.Agentes.Busqueda_anchura()
game = dependencias.Snake_game((15, 15), 5, agente_searcher)

In [14]:
# Ejecuta esta celda para ver como se comporta el agente

game.play_with_pygame()

A continuacion definimos una mezcla de los 3 agentes anteriores, para ver como se comporta en el juego.

In [14]:
agente_searcher = dependencias.Agentes.Busqueda_anchura()
agente_avoider = dependencias.Agentes.Avoid_inmediate_death()
agente_chaser = dependencias.Agentes.ChaserAgent()

deep_chouder = dependencias.Agentes.Combined_agent((agente_avoider, agente_chaser, agente_searcher), (1, 0.2, 3))
game = dependencias.Snake_game((15, 15), 5, deep_chouder)

In [25]:
# Ejecuta esta celda para ver como se comporta el agente

game.play_with_pygame()

#### Comportamiento del agente

- El comportamiento de este agente es muy bueno, consigue sobrevivir mucho tiempo y llegar a la comida de manera eficiente, además cuando se queda encerrado sigue opcupando espacio hasta generar una salida y una vez existe es capaz de escapar generando muchas huidas interesantes.

In [17]:
# lo comparamos con el agente que desarrollamos antes.
# Esta celda puede tardar un poco en ejecutarse se recomienda siplemente mirar los resultados.

dependencias.enfrentar(deep_chouder, chouder, n_partidas=100)

100%|██████████| 100/100 [00:00<00:00, 218.12it/s]

         Estadisticas          |    Agente 1     |    Agente 2    
------------------------------ | --------------- | ---------------
puntuacion_media               |   ((*78.64*))   |      32.42     
puntuacion_maxima              |    ((*119*))    |       61       
puntuacion_minima              |    ((*50*))     |       10       
movimientos_medios             |  ((*544.43*))   |     175.77     
movimientos_maximos            |   ((*1038*))    |       355      
movimientos_minimos            |    ((*325*))    |       40       
movimientos por puntuacion     |        7        |     ((*5*))    
proporcion_del_tablero_ocupada |   ((*0.27*))    |      0.16      
------------------------------ | --------------- | ---------------


Estas estaditicas muestran que deep_chouder es un agente mucho mejor que chouder, consigue sobrevivir mucho mas tiempo y llegar a mas comida. Sin embargo logicamente chouder es as eficiente y crece mas rapido.

### Conclusiones

deep_chouder es sustancialente mejor que chouder, sin cuando la partida alcanza una catidad alta de comida, surge un problema. Como todo esta tan lleno, el agente prefiere quedarse encerrado cuando en realidad claramente lo lleva a morir.

A continuacion vamos a desarrollar con busqueda en anchuera que cumpla el mismo objetivo que este a partir de otra idea que hemos tenido.

## Agentes Tail Chasser

Este agente se basa en la idea de que si el agente tiene acceso a la cola entonces puede crear un ciclo, de modo que el agente mientras no se salga del ciclo puede seguir vivo infinitamente. Parece que encontrar coida en este ciclo da problemas, pero no es el caso, para el caso mas extremo en el que tienes una comida y inmediatamente despues la cola, el agente come y se pega a la cola, pero no llega nunca a chocarse puesto que ocupar el espacio que ocupaba la cola en el espacio anterior es un movimiento valido. (El agente presente este comportamiento en la practica muchas veces), Etonces lanzareos busqueda en anchura y premiaremos los movimientos que nos permitan llegar a la cola.



<div style="text-align: center;">
    <img src="media/imagenes/aclaracion.png" style="display: inline-block; margin: auto;" width="250">
    <img src="media/imagenes/tail_chasser.png" style="display: inline-block; margin: auto;" width="400">
</div>

- *aclarar que en la imagen 1 el movimiento arriba es valido ya que la cola tambien se mueve*
*Vemos como si tienes acceso a la cola entonces no te puedes quedar encerrado*

In [18]:
tail_chasser = dependencias.Agentes.Tail_Chasser()
game = dependencias.Snake_game((15, 15), 5, tail_chasser)

In [19]:
# Ejecuta esta celda para ver como se comporta el agente

game.play_with_pygame() # Esta estrategia solo se puede ver si la serpiente es suficientemente larga por lo que tardara en mostrar coportamientos interesantes por si sola

vamos a incorporar esta estrategia a deep_chouder para ver como se comporta.

In [7]:
agente_avoider = dependencias.Agentes.Avoid_inmediate_death()
agente_chaser = dependencias.Agentes.ChaserAgent()
agente_tail_chasser = dependencias.Agentes.Tail_Chasser()
agente_searcher = dependencias.Agentes.Busqueda_anchura()

deep_loopy_looper = dependencias.Agentes.Combined_agent((agente_avoider,
                                                         agente_chaser,
                                                         agente_searcher,
                                                         agente_tail_chasser),
                                                        
                                                        (1, 0.2, 2, 1))

game = dependencias.Snake_game((15, 15), 5, deep_loopy_looper)

In [24]:
# Ejecuta esta celda para ver como se comporta el agente

game.play_with_pygame()

In [18]:
dependencias.enfrentar(deep_loopy_looper, deep_chouder, n_partidas=100)

# Esta celda tara un poco en ejecutar, se recomienda simplemente mirar los resultados.

100%|██████████| 100/100 [01:12<00:00,  1.39it/s]

         Estadisticas          |    Agente 1     |    Agente 2    
------------------------------ | --------------- | ---------------
puntuacion_media               |   ((*88.42*))   |      79.88     
puntuacion_maxima              |    ((*140*))    |       114      
puntuacion_minima              |       52        |    ((*53*))    
movimientos_medios             |  ((*633.37*))   |     548.53     
movimientos_maximos            |   ((*1206*))    |       903      
movimientos_minimos            |    ((*346*))    |       343      
movimientos por puntuacion     |        7        |     ((*7*))    
proporcion_del_tablero_ocupada |   ((*0.44*))    |      0.24      
------------------------------ | --------------- | ---------------


Deep_loopy_looper se coloca se coloca en cabeza puesto que es en general mejor que deep_chouder.

### Observacion

*Este tipo de agentes parece que mueren consistentemente cuando desovedecen la politica de tail_chasser, parece buena idea darle mucho peso, sin embargo esto puede dar lugar a bucles infinitos, que hasta hora no estan gestionados.*


<img src="media/imagenes/vacio.png" style="display: block; margin: auto;" width="200">

*Además este agente recubre aproximadamente la mitad de del tablero pero la eficiencia de este empaquetando es bastante mala, quizás podramos implementar una politica rellene mejor los espacios*

## Filler

Esta politica, se basa en premiar los movimientos que tangan partes de la serpiente adyacentes, de este modo se elegiran menos movimientos que separan a la serpiente demasiado a no ser que esto este justificado por otras politicas.



In [28]:
filler = dependencias.Agentes.Filler()

# No tiene mucho sentido a estas alturas probar los agentes por separado, vamos a generan una cobinacion.

In [1]:
import dependencias

SyntaxError: closing parenthesis ')' does not match opening parenthesis '{' on line 258 (juego_base.py, line 266)

In [8]:
agente_avoider = dependencias.Agentes.Avoid_inmediate_death()
agente_chaser = dependencias.Agentes.ChaserAgent()
agente_tail_chasser = dependencias.Agentes.Tail_Chasser()
agente_searcher = dependencias.Agentes.Busqueda_anchura()
filler = dependencias.Agentes.Filler()

filete = dependencias.Agentes.Combined_agent((           agente_avoider,
                                                         agente_chaser,
                                                         agente_searcher,
                                                         agente_tail_chasser,
                                                         filler),
                                                        
                                                        (1, 0.3, 2, 0.5, 0.3))

game = dependencias.Snake_game((15, 15), 5, filete)

In [12]:
# Ejecuta esta celda para ver como se comporta el agente

game.play_with_pygame()

In [11]:
dependencias.enfrentar(filete, deep_loopy_looper, n_partidas=10)

100%|██████████| 10/10 [00:11<00:00,  1.13s/it]

         Estadisticas          |    Agente 1     |    Agente 2    
------------------------------ | --------------- | ---------------
puntuacion_media               |   ((*97.0*))    |      87.6      
puntuacion_maxima              |       110       |    ((*115*))   
puntuacion_minima              |    ((*74*))     |       67       
movimientos_medios             |   ((*911.9*))   |      622.6     
movimientos_maximos            |   ((*1196*))    |       845      
movimientos_minimos            |    ((*706*))    |       454      
movimientos por puntuacion     |        9        |     ((*7*))    
proporcion_del_tablero_ocupada |   ((*0.43*))    |      0.39      
------------------------------ | --------------- | ---------------


Parece que filete se comporta mejor que deep_loopy_looper, pero de nuevo da problemas con tema bucles y conflictos con chasser, por lo tanto lo siguiente que haremos es implementar una politica para evitar ciclos.

<img src="media/imagenes/filete.png" style="display: block; margin: auto;" width="200">

*Lo que si que es muy prometedor es observar como se comporta el agente, claramente es campaz de gestionar mejor el espacio.*

## Cycle_detector

Esta politica no hace nada hasta que detecta que la serpiente ha completado un ciclo identico (dos subsecuecias identicas en las secuencias de movimientos), a continuacion añadimos premios con el objetivo de salir del ciclo si ocurre dejaremos de premiar de nuevo. Una vez se detecta el bucle tenemos dos maneras de continuar:

- (matar al agente) -> Premiando cualquier movimiento el agente termina movimiento, esto nos permite diseñar estrategias con riesgo de quedarse en bucle, y que no se queden en bucle, sin embargo dara lugar a partidas atipicas a las que se llegue a un bucle al principio y el agente muera.

- (premiar searcher y añadir aleatoriedad) -> Hemos observado que searcher es eficaz saliendo de bucles ya que nos llevara a una casilla que aparentemente sera segura, esto es mas costoso, veremos si merece la pena.

vamoa a implementar ambas estrategias y acompararlas.

In [26]:
import dependencias

import dependencias

agente_avoider = dependencias.Agentes.Avoid_inmediate_death()
agente_chaser = dependencias.Agentes.ChaserAgent()
agente_tail_chasser = dependencias.Agentes.Tail_Chasser()
agente_searcher = dependencias.Agentes.Busqueda_anchura()
filler = dependencias.Agentes.Filler()
cycle_detector = dependencias.Agentes.Cycle_detector(avoid_skipable_loops=False)
cycle_destroyer = dependencias.Agentes.Cycle_detector(avoid_skipable_loops=True)

senollop = dependencias.Agentes.Combined_agent(
    agentes=(agente_tail_chasser,
             agente_chaser,
             agente_avoider,
             filler,
             agente_searcher,
             cycle_destroyer), 
    weights=(10, 0.5, 0.55, 0.05, 0, 100))

fileton = dependencias.Agentes.Combined_agent(
    agentes=(agente_tail_chasser,
             agente_chaser,
             agente_avoider,
             filler,
             agente_searcher,
             cycle_detector), 
    weights=(10, 0.5, 0.55, 0.05, 0, 100))

In [30]:
dependencias.enfrentar(senollop, fileton, n_partidas=50)

100%|██████████| 50/50 [01:59<00:00,  2.38s/it]

         Estadisticas          |    Agente 1     |    Agente 2    
------------------------------ | --------------- | ---------------
puntuacion_media               |  ((*216.58*))   |     210.48     
puntuacion_maxima              |    ((*223*))    |       222      
puntuacion_minima              |    ((*202*))    |       20       
movimientos_medios             | ((*12507.58*))  |     3599.86    
movimientos_maximos            |   ((*43919*))   |      6488      
movimientos_minimos            |   ((*2896*))    |       151      
movimientos por puntuacion     |       58        |    ((*17*))    
proporcion_del_tablero_ocupada |   ((*0.96*))    |      0.94      
------------------------------ | --------------- | ---------------


## Senollop

con la misma idea que antes vamos a ajustar los hiperparametros de los agentes para conseguir el agente definitivo.

In [1]:
import dependencias

agente_avoider = dependencias.Agentes.Avoid_inmediate_death()
agente_chaser = dependencias.Agentes.ChaserAgent()
agente_tail_chasser = dependencias.Agentes.Tail_Chasser()
agente_searcher = dependencias.Agentes.Busqueda_anchura()
filler = dependencias.Agentes.Filler()
cycle_detector = dependencias.Agentes.Cycle_detector(avoid_skipable_loops=False)
cycle_destroyer = dependencias.Agentes.Cycle_detector(avoid_skipable_loops=True)

senollop = dependencias.Agentes.Combined_agent(
    agentes=(agente_tail_chasser, agente_chaser, agente_avoider, filler, agente_searcher, cycle_destroyer), weights=(10, 0.5, 0.55, 0.05, 0, 100))

game = dependencias.Snake_game((15, 15), 5, senollop)

pygame 2.6.1 (SDL 2.28.4, Python 3.12.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [8]:
# Ejecuta esta celda para ver como se comporta el agente

game.play_with_pygame()

In [23]:
game.evaluar()

100%|██████████| 100/100 [03:53<00:00,  2.33s/it]


{'puntuacion_media': np.float64(208.73),
 'puntuacion_maxima': np.int64(224),
 'puntuacion_minima': np.int64(10),
 'movimientos_medios': np.float64(3733.93),
 'movimientos_maximos': np.int64(5154),
 'movimientos_minimos': np.int64(54),
 'movimientos por puntuacion': 18,
 'proporcion_del_tablero_ocupada': np.float64(0.93)}

# Hemos Ganao?

vamos simplemente a hacer algunas pruebas con para ver los resultados finales

## Tablero 5 x 5

In [2]:
game_1 = dependencias.Snake_game((5, 5), 5, senollop)

In [3]:
# Ejecuta esta celda para ver como se comporta el agente

game_1.play_with_pygame()

In [4]:
game_1.evaluar()

 35%|███▌      | 35/100 [00:00<00:00, 170.56it/s]

SENOLLOP HA GANADO GRACIAS POR JUGAR
SENOLLOP HA GANADO GRACIAS POR JUGAR
SENOLLOP HA GANADO GRACIAS POR JUGAR
SENOLLOP HA GANADO GRACIAS POR JUGAR
SENOLLOP HA GANADO GRACIAS POR JUGAR
SENOLLOP HA GANADO GRACIAS POR JUGAR
SENOLLOP HA GANADO GRACIAS POR JUGAR
SENOLLOP HA GANADO GRACIAS POR JUGAR
SENOLLOP HA GANADO GRACIAS POR JUGAR
SENOLLOP HA GANADO GRACIAS POR JUGAR
SENOLLOP HA GANADO GRACIAS POR JUGAR
SENOLLOP HA GANADO GRACIAS POR JUGAR
SENOLLOP HA GANADO GRACIAS POR JUGAR
SENOLLOP HA GANADO GRACIAS POR JUGAR
SENOLLOP HA GANADO GRACIAS POR JUGAR
SENOLLOP HA GANADO GRACIAS POR JUGAR
SENOLLOP HA GANADO GRACIAS POR JUGAR


 70%|███████   | 70/100 [00:00<00:00, 166.90it/s]

SENOLLOP HA GANADO GRACIAS POR JUGAR
SENOLLOP HA GANADO GRACIAS POR JUGAR
SENOLLOP HA GANADO GRACIAS POR JUGAR
SENOLLOP HA GANADO GRACIAS POR JUGAR
SENOLLOP HA GANADO GRACIAS POR JUGAR
SENOLLOP HA GANADO GRACIAS POR JUGAR
SENOLLOP HA GANADO GRACIAS POR JUGAR
SENOLLOP HA GANADO GRACIAS POR JUGAR
SENOLLOP HA GANADO GRACIAS POR JUGAR


100%|██████████| 100/100 [00:00<00:00, 163.66it/s]

SENOLLOP HA GANADO GRACIAS POR JUGAR
SENOLLOP HA GANADO GRACIAS POR JUGAR
SENOLLOP HA GANADO GRACIAS POR JUGAR
SENOLLOP HA GANADO GRACIAS POR JUGAR


{'puntuacion_media': np.float64(23.68),
 'puntuacion_maxima': np.int64(25),
 'puntuacion_minima': np.int64(18),
 'movimientos_medios': np.float64(94.95),
 'movimientos_maximos': np.int64(126),
 'movimientos_minimos': np.int64(65),
 'movimientos por puntuacion': 4,
 'proporcion_del_tablero_ocupada': np.float64(0.95)}

Este tablero lo gana muy amenudo y tiene resultados optimos.

## Tablero 10 x 10

In [12]:
game_2 = dependencias.Snake_game((10, 10), 5, senollop)

In [13]:
# Ejecuta esta celda para ver como se comporta el agente

game_2.play_with_pygame()

In [15]:
game_2.evaluar()

  8%|▊         | 8/100 [00:01<00:16,  5.42it/s]

SENOLLOP HA GANADO GRACIAS POR JUGAR


 10%|█         | 10/100 [00:01<00:15,  5.78it/s]

SENOLLOP HA GANADO GRACIAS POR JUGAR


 38%|███▊      | 38/100 [00:07<00:11,  5.45it/s]

...

SENOLLOP HA GANADO GRACIAS POR JUGAR


 66%|██████▌   | 66/100 [00:12<00:06,  5.16it/s]

SENOLLOP HA GANADO GRACIAS POR JUGAR


100%|██████████| 100/100 [00:19<00:00,  5.09it/s]


{'puntuacion_media': np.float64(95.26),
 'puntuacion_maxima': np.int64(100),
 'puntuacion_minima': np.int64(84),
 'movimientos_medios': np.float64(820.57),
 'movimientos_maximos': np.int64(1153),
 'movimientos_minimos': np.int64(534),
 'movimientos por puntuacion': 9,
 'proporcion_del_tablero_ocupada': np.float64(0.95)}

El 10 x 10 tambien lo gana bastante.

## Tablero 15 x 15

Durante todo este proceso conseguimos ganar el 15 x 15 en dos ocasiones, logrando el preciado 225.

<div style="text-align: center;">
    <img src="media/imagenes/win_1.png" style="display: inline-block; margin: auto;" width="400">
    <img src="media/imagenes/win_2.png" style="display: inline-block; margin: auto;" width="400">
</div>
 

## 17 x 15

por ultimo vamos a evaluar el comportamiento en el problema que lo inició todo, esas clases de programación aburridas donde necesibas algo que hacer, y el humilde snake de google (15 x 17) te ayudaba a pasar el rato.

<img src="media/imagenes/google.png" style="display: block; margin: auto;" width="200">

In [19]:
game_3 = dependencias.Snake_game((15, 17), 5, senollop)

In [20]:
# Ejecuta esta celda para ver como se comporta el agente

game_3.play_with_pygame()

In [23]:
game_3.evaluar()

100%|██████████| 100/100 [05:47<00:00,  3.47s/it]


{'puntuacion_media': np.float64(239.12),
 'puntuacion_maxima': np.int64(252),
 'puntuacion_minima': np.int64(14),
 'movimientos_medios': np.float64(4539.67),
 'movimientos_maximos': np.int64(6851),
 'movimientos_minimos': np.int64(115),
 'movimientos por puntuacion': 19,
 'proporcion_del_tablero_ocupada': np.float64(0.94)}

# Playgrond

Aqui puedes jugar a crear tus propios agentes y ver como se comportan en el juego, los parametros que se dan son los de senollop puedes probar:

- crear agentes
- llenar el mapa de comida
- cambiar el tamaño del tablero (muy grande muy pequeño)

In [36]:
WEIGHTS =(10, 0.5, 0.55, 0.05, 0, 100)
SIZE = (15, 15)
N_FOODS = 5
N_EVALUACIONES = 100

In [37]:
import dependencias

agente_avoider = dependencias.Agentes.Avoid_inmediate_death()
agente_chaser = dependencias.Agentes.ChaserAgent()
agente_tail_chasser = dependencias.Agentes.Tail_Chasser()
agente_searcher = dependencias.Agentes.Busqueda_anchura()
filler = dependencias.Agentes.Filler()
cycle_detector = dependencias.Agentes.Cycle_detector(avoid_skipable_loops=False)
cycle_destroyer = dependencias.Agentes.Cycle_detector(avoid_skipable_loops=True)

agente_custom = dependencias.Agentes.Combined_agent(
    agentes=(agente_tail_chasser, agente_chaser, agente_avoider, filler, agente_searcher, cycle_destroyer), weights=WEIGHTS)

game = dependencias.Snake_game(SIZE, N_FOODS, senollop)

In [38]:
game.play_with_pygame()

In [ ]:
game.evaluar()

<img src="media/imagenes/Logo.jpg" style="display: block; margin: auto;" width="200">